In [33]:
from minio import Minio

# Create client with access and secret key.
client = Minio("oss.geocodes.ncsa.illinois.edu:443", "worldsbestaccesskey", "worldsbestsecretkey")

In [53]:
path = f"milled/iris/001dcdf99c383571c0479c00def62ed953c83a92.rdf"
response = client.get_object("yybucket", path)
#print(response.data)
rdf = response.data

path = f"summoned/iris/001dcdf99c383571c0479c00def62ed953c83a92.jsonld"
response = client.get_object("yybucket", path)
jsonld = response.data

path = f"graphs/latest/iris_release.nq"
response = client.get_object("yybucket", path)
nq = response.data

f = open("shape.ttl", "r")
#print(f.read())

In [60]:
def validate_graph(graph, shacl_path=None, format='nquads'):
    """Validate the current graph with a SHACL schema.

    Uses default schema if not supplied.
    """
    if shacl_path:
        with open(shacl_path, 'r', encoding='utf-8') as f:
            shacl = f.read()
    else:
        shacl = resource_string('renku', 'data/shacl_shape.json')

    return validate(
        graph,
        shacl_graph=shacl,
        inference='rdfs',
        meta_shacl=True,
        debug=False,
        #data_graph_format=format,
        #shacl_graph_format='json-ld',
        advanced=True
    )

In [65]:
validate_graph(rdf, 'shape.ttl', 'rdfs')

(True,
 <Graph identifier=N86f2ab9e5b604d3d9b565ecf95735050 (<class 'rdflib.graph.Graph'>)>,
 'Validation Report\nConforms: True\n')

In [67]:
validate_graph(rdf, 'shape.ttl', 'rdfs')

(False,
 <Graph identifier=Nf30f2cbfc25d48d5b1a76d4fdaf366ee (<class 'rdflib.graph.Graph'>)>,
 'Validation Report\nConforms: False\nResults (5):\nConstraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: [ sh:lessThan schema1:deathDate ; sh:message Literal("YY TEST") ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:path schema1:birthDate ]\n\tFocus Node: <https://ds.iris.edu/ds/products/eventplots/>\n\tResult Path: schema1:birthDate\n\tMessage: YY TEST\nConstraint Violation in MinLengthConstraintComponent (http://www.w3.org/ns/shacl#MinLengthConstraintComponent):\n\tSeverity: sh:Violation\n\tSource Shape: ecgeo:datasetNameProperty\n\tFocus Node: <https://ds.iris.edu/ds/products/eventplots/>\n\tValue Node: Literal("Maps, record sections & other plots for M6.0+ events")\n\tResult Path: schema1:name\n\tMessage: Name is required; Dataset names must have at least 15 characters\nValidation Res